In [35]:
import numpy as np
import pandas as pd
import re

In [7]:
data=pd.read_excel("./Data/한국마사회_경주상세정보(제주_부경)_20230720.xlsx")

In [8]:
unicol_list = []
for col in data.columns:
    if data[1:][col].nunique()<=1:
        unicol_list.append(col)
drop_data = data.drop(unicol_list,axis=1) 

In [9]:
drop_data["WG"] = drop_data["WG_HR"].str.extract(r'(\d+)(?=\()')
drop_data["WG"][0]="마체중"

drop_data["WC"] = drop_data["WG_HR"].str.extract(r'\((.*?)\)')
drop_data["WC"][0] = "체중증감"

drop_data["WC"][1:] = pd.to_numeric(drop_data["WC"][1:])
drop_data["WG"][1:] = pd.to_numeric(drop_data["WG"][1:])

drop_data = drop_data.drop(["WG_HR"],axis=1)

In [ ]:
#한글 이름을 열로 바꿔줌
drop_data.columns=drop_data.iloc[0]
drop_data=drop_data[1:]
drop_data

In [31]:
#1착, 2착 횟수가 높다고 좋은 것이 아니라 승률이 높아야 좋다고 생각하기에, 경주마, 기수, 조교수, 위탁 각각의 승률을 새로운 열에 추가해주었다.
drop_data['경주마통산1착승률']=drop_data['경주마통산1착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마통산2찬승률']=drop_data['경주마통산2착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마최근1년1착승률']=drop_data['경주마최근1년1착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)
drop_data['경주마최근1년2착승률']=drop_data['경주마최근1년2착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)

drop_data['기수통산1착승률']=drop_data['기수통산1착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수통산2착승률']=drop_data['기수통산2착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수최근1년간1착승률']=drop_data['기수최근1년간1착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)
drop_data['기수최근1년간2착승률']=drop_data['기수최근1년간2착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)

drop_data['조교사통산1착승률']=drop_data['조교사통산1착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사통산2착승률']=drop_data['조교사통산2착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사최근1년간1착승률']=drop_data['조교사최근1년간1착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)
drop_data['조교사최근1년간2착승률']=drop_data['조교사최근1년간2착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)

drop_data['조교사위탁경주마1위승률']=drop_data['조교사위탁경주마1위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)
drop_data['조교사위탁경주마2위승률']=drop_data['조교사위탁경주마2위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)


C:\Users\wjswp\AppData\Local\Temp\ipykernel_5392\3409290213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_data['경주마통산1착승률']=drop_data['경주마통산1착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
C:\Users\wjswp\AppData\Local\Temp\ipykernel_5392\3409290213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_data['경주마통산2찬승률']=drop_data['경주마통산2착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
C:\Users\wjswp\AppData\Local\Temp\ipykernel_5392\3409290213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

In [47]:
#주로상태(함수율) 데이터가 포화(18%)로 되어 있는 것을 포화, 18로 바꿔주었다.
drop_data[['주로상태','주로상태_함수율']]=drop_data['주로상태(함수율)'].str.extract(r'(.*?) \((\d+)%\)')
drop_data=drop_data.drop('주로상태(함수율)',axis=1)

C:\Users\wjswp\AppData\Local\Temp\ipykernel_5392\3108640001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_data[['주로상태','주로상태_함수율']]=drop_data['주로상태(함수율)'].str.extract(r'(.*?) \((\d+)%\)')


In [48]:
drop_data

,경주일자,경주번호,경주거리,경주등급,날씨,착순,출주번호,마명,마번,마필연령,...,기수최근1년간1착승률,기수최근1년간2착승률,조교사통산1착승률,조교사통산2착승률,조교사최근1년간1착승률,조교사최근1년간2착승률,조교사위탁경주마1위승률,조교사위탁경주마2위승률,주로상태,주로상태_함수율
1,20230701,1,800,제6등급,흐림,1,9,한백제왕,3101848,3,...,0.154964,0.121065,0.152249,0.121543,0.149533,0.099688,0.245902,0.125683,포화,18
2,20230701,1,800,제6등급,흐림,2,2,명의로운,3102087,3,...,0.064677,0.074627,0.096586,0.104722,0.043732,0.069971,0.048973,0.127962,포화,18
3,20230701,1,800,제6등급,흐림,3,3,번개의꿈,3101489,4,...,0.158986,0.124424,0.082014,0.101251,0.048682,0.099391,0.109116,0.104972,포화,18
4,20230701,1,800,제6등급,흐림,4,6,고차원,3103528,2,...,0.071979,0.092545,0.082014,0.101251,0.048682,0.099391,0.109116,0.104972,포화,18
5,20230701,1,800,제6등급,흐림,5,1,위너고,3102911,3,...,0.065693,0.072993,0.093358,0.099067,0.0475,0.0775,0.084081,0.098655,포화,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29588,20190111,9,900,한4등급,흐림,6,8,백년친구,1017787,3,...,0.0,0.0,0.116165,0.117293,0.102362,0.091864,0.117873,0.122998,양호,8
29589,20190111,9,900,한4등급,흐림,7,3,미래제주,1017443,4,...,0.158986,0.124424,0.102273,0.107801,0.125,0.111111,0.08775,0.125109,양호,8
29590,20190111,9,900,한4등급,흐림,8,1,남단여제,1017686,3,...,0.0,0.0,0.094704,0.102217,0.117143,0.142857,0.0954,0.088586,양호,8
29591,20190111,9,900,한4등급,흐림,9,4,섬문화,1017765,4,...,0.094421,0.077253,0.078531,0.093028,0.091667,0.075,0.072055,0.095424,양호,8
